# Web Scrape - Hoje em Dia

Objetivo: extrair a headline, descrição e link das principais notícias do portal Hoje em Dia afim de identificar padrões presentes em notícias consideradas verdadeiras. A fonte é um portal ligado a Rede Record.

Fonte: http://hojeemdia.com.br/primeiro-plano/pol%C3%ADtica

## Importe das bibliotecas

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm

## Definição das funções de parse da URL

Função que retorna o HTML da página

In [3]:
def parse_url(url):
    r = requests.get(url)
    html = BeautifulSoup(r.text, 'html5lib')
    
    return html

Função que extrai título, link, data e resumo do post. Como cada site possui uma estrutura diferente, não é possível generalizar essa função.

In [4]:
def get_hd_posts(url, pages=1):
    post_data = []
    
    html = parse_url(url)
    posts_hd = html.find_all('li', 'post')
    
    for i, post in enumerate(posts_hd):
        title = post.find('div', 'title').find('a').get_text()
        link = post.find('div', 'title').find('a').get('href')
        date = post.find('div', 'time').get_text().replace('\n', '')
        excerpt = title
        post_data.append([title, link, date, excerpt])
        
    if pages > 1:
        for p in tqdm(range(2, pages)):
            html = parse_url(url + '?page=' + str(p))
            posts_hd = html.find_all('li', 'post')

            for i, post in enumerate(posts_hd):
                title = post.find('div', 'title').find('a').get_text()
                link = post.find('div', 'title').find('a').get('href')
                date = post.find('div', 'time').get_text().replace('\n', '')
                excerpt = title
                post_data.append([title, link, date, excerpt])
    
    return np.array(post_data)

## Criação e tratamento do dataframe

In [6]:
posts_hd = get_hd_posts('http://hojeemdia.com.br/primeiro-plano/pol%C3%ADtica', pages=10)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  2.71it/s]


Verifica a qualidade dos dados gerados

In [7]:
dataframe_hd = pd.DataFrame(posts_hd, columns=['title', 'url', 'date', 'excerpt'])

In [8]:
dataframe_hd.head()

,title,url,date,excerpt
0,Professores da UFMG realizam ato em defesa ao ...,/primeiro-plano/professores-da-ufmg-realizam-a...,10:17,Professores da UFMG realizam ato em defesa ao ...
1,Câmara inicia debate da reforma da Previdência...,/primeiro-plano/pol%C3%ADtica/c%C3%A2mara-inic...,22:02,Câmara inicia debate da reforma da Previdência...
2,Temer viaja para Argentina com expectativa de ...,/primeiro-plano/temer-viaja-para-argentina-com...,21:20,Temer viaja para Argentina com expectativa de ...
3,"Temer convoca reunião com Rodrigo Maia, minist...",/primeiro-plano/pol%C3%ADtica/temer-convoca-re...,19:32,"Temer convoca reunião com Rodrigo Maia, minist..."
4,Lula defende acordos com oposição se voltar ao...,/primeiro-plano/pol%C3%ADtica/lula-defende-aco...,16:14,Lula defende acordos com oposição se voltar ao...


In [9]:
dataframe_hd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 4 columns):
title      180 non-null object
url        180 non-null object
date       180 non-null object
excerpt    180 non-null object
dtypes: object(4)
memory usage: 5.7+ KB


Exporta pra CSV

In [ ]:
dataframe_hd.to_csv('hojeemdia.csv', encoding='utf-8')